# Imports

In [1]:
from glob import glob
import numpy as np
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"]="3"
from skimage import io
from skimage.transform import resize
import tensorflow as tf
from tensorflow.keras.applications.xception import preprocess_input as Optimized_preprocess
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input as VGG16_preprocess
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input as MobileNet_preprocess
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input as ResNet_preprocess
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.utils import Sequence
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
import imgaug as ia
import imgaug.augmenters as iaa
import matplotlib.pyplot as plt
from tensorflow.keras import backend as K
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd
import seaborn as sns; sns.set()

tf.get_logger().setLevel('ERROR')


In [2]:
data_set = ['Train', 'Validation', 'Test']
genders = ['Female', 'Male']

paths = './data/{dataSet}/{gender}*/*.jpg'

partition = {'train':      {'img': [], 'label': []},
             'validation': {'img': [], 'label': []},
             'test':       {'img': [], 'label': []},
             }

for dataSet in data_set:
  for gender in genders:
    for path in glob(paths.format(dataSet=dataSet, gender=gender)):
      partition[dataSet.lower()]['label'].append(gender)
      partition[dataSet.lower()]['img'].append(path)

masks = []
texture = []
for path in glob('./data/masks/*'):
  if os.path.isdir(path):
    for tex in glob(os.path.join(path, '*')):
      texture.append(tex)
  else:
    masks.append(path)

In [3]:
function_none = lambda x: x
function_one_hot = lambda x: to_categorical(x, 2)

def model_optimized():
  inputs = layers.Input((224, 224, 3))
  x = layers.ZeroPadding2D(padding=((0, 1), (0, 1)), name='conv1_pad')(inputs)
  x = layers.Conv2D(32, (3, 3), padding='valid', use_bias=False, strides=(2, 2), name='conv1')(x)
  x = layers.BatchNormalization(name='conv1_bn')(x)
  x = layers.ReLU(10., name='conv1_relu')(x)

  id = 0
  id += 1
  x = layers.SeparableConv2D(64, (3, 3), padding='same',
                             strides=(1, 1), use_bias=False,
                             name='conv_dw_{}'.format(id))(x)
  x = layers.BatchNormalization(name='conv_dw_{}_bn'.format(id))(x)
  x = layers.ReLU(10., name='conv_dw_{}_relu'.format(id))(x)

  id += 1
  x = layers.SeparableConv2D(128, (3, 3), padding='valid',
                             strides=(2, 2), use_bias=False,
                             name='conv_dw_{}'.format(id))(x)
  x = layers.BatchNormalization(name='conv_dw_{}_bn'.format(id))(x)
  x = layers.ReLU(10., name='conv_dw_{}_relu'.format(id))(x)

  id += 1
  x = layers.SeparableConv2D(128, (3, 3), padding='same',
                             strides=(1, 1), use_bias=False,
                             name='conv_dw_{}'.format(id))(x)
  x = layers.BatchNormalization(name='conv_dw_{}_bn'.format(id))(x)
  x = layers.ReLU(10., name='conv_dw_{}_relu'.format(id))(x)

  id += 1
  x = layers.SeparableConv2D(256, (3, 3), padding='valid',
                             strides=(2, 2), use_bias=False,
                             name='conv_dw_{}'.format(id))(x)
  x = layers.BatchNormalization(name='conv_dw_{}_bn'.format(id))(x)
  x = layers.ReLU(10., name='conv_dw_{}_relu'.format(id))(x)

  id += 1
  x = layers.SeparableConv2D(256, (3, 3), padding='same',
                             strides=(1, 1), use_bias=False,
                             name='conv_dw_{}'.format(id))(x)
  x = layers.BatchNormalization(name='conv_dw_{}_bn'.format(id))(x)
  x = layers.ReLU(10., name='conv_dw_{}_relu'.format(id))(x)

  id += 1
  x = layers.SeparableConv2D(512, (3, 3), padding='valid',
                             strides=(2, 2), use_bias=False,
                             name='conv_dw_{}'.format(id))(x)
  x = layers.BatchNormalization(name='conv_dw_{}_bn'.format(id))(x)
  x = layers.ReLU(10., name='conv_dw_{}_relu'.format(id))(x)

  id += 1
  x = layers.SeparableConv2D(512, (3, 3), padding='same',
                             strides=(1, 1), use_bias=False,
                             name='conv_dw_{}'.format(id))(x)
  x = layers.BatchNormalization(name='conv_dw_{}_bn'.format(id))(x)
  x = layers.ReLU(10., name='conv_dw_{}_relu'.format(id))(x)

  id += 1
  x = layers.SeparableConv2D(512, (3, 3), padding='same',
                             strides=(1, 1), use_bias=False,
                             name='conv_dw_{}'.format(id))(x)
  x = layers.BatchNormalization(name='conv_dw_{}_bn'.format(id))(x)
  x = layers.ReLU(10., name='conv_dw_{}_relu'.format(id))(x)

  id += 1
  x = layers.SeparableConv2D(512, (3, 3), padding='same',
                             strides=(1, 1), use_bias=False,
                             name='conv_dw_{}'.format(id))(x)
  x = layers.BatchNormalization(name='conv_dw_{}_bn'.format(id))(x)
  x = layers.ReLU(10., name='conv_dw_{}_relu'.format(id))(x)


  id += 1
  x = layers.SeparableConv2D(512, (3, 3), padding='same',
                             strides=(1, 1), use_bias=False,
                             name='conv_dw_{}'.format(id))(x)
  x = layers.BatchNormalization(name='conv_dw_{}_bn'.format(id))(x)
  mask = layers.ReLU(10., name='conv_dw_{}_relu'.format(id))(x)

  id += 1
  x = layers.SeparableConv2D(512, (3, 3), padding='same',
                             strides=(1, 1), use_bias=False,
                             name='conv_dw_{}'.format(id))(mask)
  x = layers.BatchNormalization(name='conv_dw_{}_bn'.format(id))(x)
  x = layers.ReLU(10., name='conv_dw_{}_relu'.format(id))(x)

  id += 1
  x = layers.SeparableConv2D(1024, (3, 3), padding='valid',
                             strides=(2, 2), use_bias=False,
                             name='conv_dw_{}'.format(id))(x)
  x = layers.BatchNormalization(name='conv_dw_{}_bn'.format(id))(x)
  x = layers.ReLU(10., name='conv_dw_{}_relu'.format(id))(x)

  id += 1
  x = layers.SeparableConv2D(1024, (3, 3), padding='same',
                             strides=(1, 1), use_bias=False,
                             name='conv_dw_{}'.format(id))(x)
  x = layers.BatchNormalization(name='conv_dw_{}_bn'.format(id))(x)
  gender = layers.ReLU(10., name='conv_dw_{}_relu'.format(id))(x)

  mask = layers.GlobalAvgPool2D()(mask)
  gender = layers.GlobalAvgPool2D()(gender)

  mask = layers.Dropout(0.3)(mask)
  gender = layers.Dropout(0.3)(gender)

  mask = layers.Dense(1, activation='sigmoid', name='mask')(mask)
  gender = layers.Dense(1, activation='sigmoid', name='gender')(gender)

  model = Model(inputs=inputs, outputs=[gender, mask], name='optimized')

  model.compile('adam', {'gender': 'binary_crossentropy',
                         'mask': 'binary_crossentropy'},
                metrics=['acc'])

  return model, Optimized_preprocess, function_none

def model_VGG16():
  inputs = layers.Input((224, 224, 3))
  vgg16 = VGG16(include_top=False, weights="imagenet", input_tensor=inputs, pooling='avg')
  vgg16.trainable = False

  x = layers.Dropout(0.3)(vgg16.outputs[0])
  x = layers.Dense(512, 'relu')(x)

  gender = layers.Dense(2, activation='softmax', name='gender')(x)
  mask = layers.Dense(2, activation='softmax', name='mask')(x)

  model = Model(inputs, outputs=[gender, mask], name='vgg10006')

  model.compile('adam', {'gender': 'categorical_crossentropy',
                         'mask': 'categorical_crossentropy'},
                metrics=['acc'])

  return model, VGG16_preprocess, function_one_hot

def model_MobileNet():
  inputs = layers.Input((224, 224, 3))
  mobilenet = MobileNet(include_top=False, weights="imagenet", input_tensor=inputs, pooling='avg')
  mobilenet.trainable = False

  x = layers.Dropout(0.3)(mobilenet.outputs[0])
  x = layers.Dense(1024, 'relu')(x)

  gender = layers.Dense(1, activation='sigmoid', name='gender')(x)
  mask = layers.Dense(1, activation='sigmoid', name='mask')(x)

  model = Model(inputs, outputs=[gender, mask], name='mobilenet')

  model.compile('adam', {'gender': 'mse',
                         'mask': 'mse'},
                metrics=['acc'])

  return model, MobileNet_preprocess, function_none

def model_ResNet():
  inputs = layers.Input((224, 224, 3))
  resnet = ResNet50(include_top=False, weights="imagenet", input_tensor=inputs, pooling='avg')
  resnet.trainable = False

  x = layers.Dropout(0.3)(resnet.outputs[0])
  x = layers.Dense(1024, 'relu')(x)

  gender = layers.Dense(1, activation='sigmoid', name='gender')(x)
  mask = layers.Dense(1, activation='sigmoid', name='mask')(x)

  model = Model(inputs, outputs=[gender, mask], name='resnet')

  model.compile('adam', {'gender': 'binary_crossentropy',
                         'mask': 'binary_crossentropy'},
                metrics=['acc'])

  return model, ResNet_preprocess, function_none


models = {'optimized': model_optimized,
          'vgg16': model_VGG16,
          'mobilenet': model_MobileNet,
          'resnet': model_ResNet}

In [4]:
def plot_history(history, title):

  losses = list(np.copy(history.history['loss']))
  # accs = list(np.copy(history.history['acc']))
  hue = ['train'] * len(history.history['loss'])

  losses.extend(history.history['val_loss'])
  # accs.extend(history.history['val_acc'])
  hue.extend(['val'] * len(history.history['val_loss']))

  epoch = list(np.copy(history.epoch))
  epoch.extend(history.epoch)

  to_plot = pd.DataFrame(dict(
    epochs=epoch,
    losses=losses,
    dataset=hue,
    ))

  losses = list(np.copy(history.history['gender_loss']))
  hue = ['gender train loss'] * len(history.history['loss'])
  epoch = list(np.copy(history.epoch))

  losses.extend(list(np.copy(history.history['mask_loss'])))
  hue.extend(['mask train  loss'] * len(history.history['loss']))
  epoch.extend(list(np.copy(history.epoch)))

  losses.extend(list(np.copy(history.history['val_gender_loss'])))
  hue.extend(['gender val  loss'] * len(history.history['loss']))
  epoch.extend(list(np.copy(history.epoch)))

  losses.extend(list(np.copy(history.history['val_mask_loss'])))
  hue.extend(['mask val  loss'] * len(history.history['loss']))
  epoch.extend(list(np.copy(history.epoch)))

  to_plot_loss_class = pd.DataFrame(dict(
    epochs=epoch,
    losses=losses,
    dataset=hue
    ))

  acc = list(np.copy(history.history['gender_acc']))
  hue = ['gender train acc'] * len(history.history['loss'])
  epoch = list(np.copy(history.epoch))

  acc.extend(list(np.copy(history.history['mask_acc'])))
  hue.extend(['mask train acc'] * len(history.history['loss']))
  epoch.extend(list(np.copy(history.epoch)))

  acc.extend(list(np.copy(history.history['val_gender_acc'])))
  hue.extend(['gender val acc'] * len(history.history['loss']))
  epoch.extend(list(np.copy(history.epoch)))

  acc.extend(list(np.copy(history.history['val_mask_acc'])))
  hue.extend(['mask val acc'] * len(history.history['loss']))
  epoch.extend(list(np.copy(history.epoch)))

  to_plot_acc = pd.DataFrame(dict(
    epochs=epoch,
    accuracy=acc,
    dataset=hue
    ))

  ax = sns.relplot(x="epochs", y="losses", hue="dataset", kind="line",
                   ci="sd", data=to_plot)
  ax.set(title=title)
  plt.show()

  ax = sns.relplot(x="epochs", y="losses", hue="dataset", kind="line",
                   ci="sd", data=to_plot_loss_class)
  ax.set(title=title)
  plt.show()

  ax = sns.relplot(x="epochs", y="accuracy", hue="dataset", kind="line",
                   ci="sd", data=to_plot_acc)
  ax.set(title=title)
  plt.show()

In [5]:
models_save = {}
for name_model, model_function in models.items():
    if name_model == 'optimized':
        epochs = [60]
        batchs = [16]
    else:
        epochs = [20, 60, 200]
        batchs = [8, 16, 32]

    for epoch in epochs:
        batch = 16
        filename_best_model = './models/model_{}_E{}_B{}.h5'.format(name_model, epoch, batch)
        models_save['{}_{}_{}'.format(name_model, epoch, batch)] = filename_best_model

In [6]:
def test_models(paths, labels):
    imgs_test = []
    labels_test = {'gender': [], 'mask': []}
    for path, gender in zip(paths, labels):
        labels_test['gender'].append(int(gender == 'Female'))
        labels_test['mask'].append(int(os.path.dirname(path).endswith('m')))
        imgs_test.append(resize(io.imread(path), [224,224])*255)

    imgs_test = np.array(imgs_test)
    labels_test['gender'] = np.array(labels_test['gender'])
    labels_test['mask'] = np.array(labels_test['mask'])

    for name, path_h5 in models_save.items():
        K.clear_session()
        model_name, epochs, batchs = name.split('_')
        model, preprocess, function_label = models[model_name]()
        model.load_weights(path_h5)

        img_test_for_model = preprocess(np.copy(imgs_test))

        labels_test_for_model = {}
        labels_test_for_model['gender'] = function_label(np.copy(labels_test['gender']))
        labels_test_for_model['mask'] = function_label(np.copy(labels_test['mask']))

        score = model.evaluate(img_test_for_model, labels_test_for_model, verbose=1)
        print('{} ; Epochs: {} ; Batchs: {}'. format(model_name, epochs, batchs))
        print('Test loss:', score[0])
        print('Test loss gender: {}\nTest loss mask: {}'.format(score[1], score[2]))
        print('Test accuracy gender: {}\nTest accuracy mask: {}'.format(score[3], score[4]))
        print("CNN Error: %.2f%%" % (100-(score[-1] + score[-2])*100/2))
        print()

In [7]:
paths =  partition['validation']['img']
labels =  partition['validation']['label']

print('Data set Validation')
test_models(paths, labels)

Data set Validation
1000/1000 [==============================] - 5s 5ms/sample - loss: 0.0852 - gender_loss: 0.0975 - mask_loss: 0.0028 - gender_acc: 0.9740 - mask_acc: 0.9990
optimized ; Epochs: 60 ; Batchs: 16
Test loss: 0.08522054510749877
Test loss gender: 0.097472183406353
Test loss mask: 0.0028326523024588823
Test accuracy gender: 0.9739999771118164
Test accuracy mask: 0.9990000128746033
CNN Error: 1.35%

1000/1000 [==============================] - 7s 7ms/sample - loss: 0.2843 - gender_loss: 0.2570 - mask_loss: 0.0365 - gender_acc: 0.8960 - mask_acc: 0.9830
vgg16 ; Epochs: 20 ; Batchs: 16
Test loss: 0.2842529745101929
Test loss gender: 0.2570410668849945
Test loss mask: 0.036462169140577316
Test accuracy gender: 0.8960000276565552
Test accuracy mask: 0.9829999804496765
CNN Error: 6.05%

1000/1000 [==============================] - 5s 5ms/sample - loss: 0.2885 - gender_loss: 0.2472 - mask_loss: 0.0511 - gender_acc: 0.8960 - mask_acc: 0.9840
vgg16 ; Epochs: 60 ; Batchs: 16
Test lo

/home/mvaldi/anaconda3/lib/python3.7/site-packages/keras_applications/mobilenet.py:207: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


1000/1000 [==============================] - 2s 2ms/sample - loss: 0.5000 - gender_loss: 0.5117 - mask_loss: 7.6822e-29 - gender_acc: 0.5000 - mask_acc: 1.0000
mobilenet ; Epochs: 20 ; Batchs: 16
Test loss: 0.5
Test loss gender: 0.51171875
Test loss mask: 7.68221676971986e-29
Test accuracy gender: 0.5
Test accuracy mask: 1.0
CNN Error: 25.00%

1000/1000 [==============================] - 2s 2ms/sample - loss: 0.0726 - gender_loss: 0.0779 - mask_loss: 7.9271e-05 - gender_acc: 0.9000 - mask_acc: 1.0000
mobilenet ; Epochs: 60 ; Batchs: 16
Test loss: 0.07257098680734635
Test loss gender: 0.07785825431346893
Test loss mask: 7.927054684842005e-05
Test accuracy gender: 0.8999999761581421
Test accuracy mask: 1.0
CNN Error: 5.00%

1000/1000 [==============================] - 2s 2ms/sample - loss: 0.0797 - gender_loss: 0.0847 - mask_loss: 2.8046e-07 - gender_acc: 0.8980 - mask_acc: 1.0000
mobilenet ; Epochs: 200 ; Batchs: 16
Test loss: 0.07970391818881035
Test loss gender: 0.08466039597988129
Te

In [8]:
paths =  partition['test']['img']
labels =  partition['test']['label']

print('Data set Test')
test_models(paths, labels)

Data set Test
792/792 [==============================] - 2s 2ms/sample - loss: 0.3298 - gender_loss: 0.2818 - mask_loss: 0.0448 - gender_acc: 0.9028 - mask_acc: 0.9861
optimized ; Epochs: 60 ; Batchs: 16
Test loss: 0.3298245887460206
Test loss gender: 0.28176262974739075
Test loss mask: 0.04483463615179062
Test accuracy gender: 0.9027777910232544
Test accuracy mask: 0.9861111044883728
CNN Error: 5.56%

792/792 [==============================] - 5s 6ms/sample - loss: 0.4219 - gender_loss: 0.3859 - mask_loss: 0.0358 - gender_acc: 0.8144 - mask_acc: 0.9861
vgg16 ; Epochs: 20 ; Batchs: 16
Test loss: 0.42192662966371786
Test loss gender: 0.3858739733695984
Test loss mask: 0.03584912046790123
Test accuracy gender: 0.814393937587738
Test accuracy mask: 0.9861111044883728
CNN Error: 9.97%

792/792 [==============================] - 4s 5ms/sample - loss: 0.4281 - gender_loss: 0.3743 - mask_loss: 0.0533 - gender_acc: 0.8245 - mask_acc: 0.9886
vgg16 ; Epochs: 60 ; Batchs: 16
Test loss: 0.42810471